In [ ]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download anasmohammedtahir/covidqu

100% 1.15G/1.15G [00:43<00:00, 37.4MB/s]
100% 1.15G/1.15G [00:43<00:00, 28.4MB/s]


In [ ]:
! unzip "/content/covidqu.zip"

Streaming output truncated to the last 5000 lines.
  inflating: Lung Segmentation Data/Lung Segmentation Data/Val/Non-COVID/lung masks/non_COVID (10835).png  
  inflating: Lung Segmentation Data/Lung Segmentation Data/Val/Non-COVID/lung masks/non_COVID (10836).png  
  inflating: Lung Segmentation Data/Lung Segmentation Data/Val/Non-COVID/lung masks/non_COVID (10837).png  
  inflating: Lung Segmentation Data/Lung Segmentation Data/Val/Non-COVID/lung masks/non_COVID (10838).png  
  inflating: Lung Segmentation Data/Lung Segmentation Data/Val/Non-COVID/lung masks/non_COVID (10839).png  
  inflating: Lung Segmentation Data/Lung Segmentation Data/Val/Non-COVID/lung masks/non_COVID (10840).png  
  inflating: Lung Segmentation Data/Lung Segmentation Data/Val/Non-COVID/lung masks/non_COVID (10841).png  
  inflating: Lung Segmentation Data/Lung Segmentation Data/Val/Non-COVID/lung masks/non_COVID (10843).png  
  inflating: Lung Segmentation Data/Lung Segmentation Data/Val/Non-COVID/lung masks/n

# ***Reading the data and extracting the features***

In [ ]:
from keras.layers import Dense, Activation, Dropout, Input, BatchNormalization
from keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import keras.backend as K
import numpy as np
from sklearn.metrics import accuracy_score
from keras.models import Model
import glob
import cv2
import numpy as np
from sklearn.utils import shuffle
import collections

In [ ]:
impath = glob.glob("Lung Segmentation Data/Lung Segmentation Data/Train/*")


In [ ]:
features=[]
label=[]
adj=[]
k=0
for i in range(len(impath)):
  f=glob.glob(impath[i]+"/*")
  #print(f)
  #for h in f:
  #print(f[1])
  x= glob.glob(f[1]+"/*.png")
  
  j=0
 
  for p in x:
      j=j+1
      
      img = cv2.imread(p)
      #print(img.shape)
      #img /= 255
      img=cv2.resize(img,(128,128))
      fea = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      features.append(fea)
      label.append(k)
      
  k=k+1

In [ ]:
impath = glob.glob("Lung Segmentation Data/Lung Segmentation Data/Test/*")

In [ ]:
k=0
for i in range(len(impath)):
  f=glob.glob(impath[i]+"/*")
  #print(f)
  #for h in f:
  #print(f[1])
  x= glob.glob(f[1]+"/*.png")
  
  j=0
 
  for p in x:
      j=j+1
      
      img = cv2.imread(p)
      #img /= 255
      img=cv2.resize(img,(128,128))
      fea = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      features.append(fea)
      label.append(k)
      
  k=k+1

In [ ]:
features = np.asarray(features)
feat=[]
for i in range(len(features)):
  feat.append(np.reshape(features[i],(128*128)))

In [ ]:
features.shape

(28503, 128, 128)

In [ ]:
print(len(feat))

28503


In [ ]:
MAX_LEN = 1000

In [ ]:
fts = feat.copy()
lbls = label.copy()

In [ ]:
fts , lbls = shuffle(fts,lbls)

In [ ]:

fts = fts[:MAX_LEN]
lbls = lbls[:MAX_LEN]

In [ ]:
fts, lbls = np.asarray(fts), np.asarray(lbls)

In [ ]:
fts.shape

(1000, 16384)

In [ ]:
lbls.shape

(1000,)

In [ ]:
print(len(fts))

1000


In [ ]:
fts[0].shape

(16384,)

In [ ]:

counter = collections.Counter(lbls)
counter

Counter({0: 316, 1: 331, 2: 353})

In [ ]:
fts[13]

array([19, 17, 15, ...,  0,  0,  3], dtype=uint8)

# ***Applying feature reduction using autoencoders***

In [ ]:

# this is our input placeholder
input_img = Input(shape=(16384,))
# "encoded" is the encoded representation of the input
encoded = Dense(2048, activation='relu')(input_img)
encoded= tf.keras.layers.BatchNormalization()(encoded)
encoded = Dropout(0.2)(encoded)
encoded = Dense(1024, activation='sigmoid')(encoded)
encoded= tf.keras.layers.BatchNormalization()(encoded)
encoded = Dropout(0.5)(encoded)
encoded = Dense(1024, activation='relu')(encoded)
encoded = Dropout(0.2)(encoded)
encoded= tf.keras.layers.BatchNormalization()(encoded)
encoded = Dense(1024, activation='sigmoid')(encoded)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(2048, activation='relu')(encoded)
decoded= tf.keras.layers.BatchNormalization()(decoded)
decoded = Dropout(0.2)(decoded)
decoded = Dense(1024, activation='relu')(decoded)
decoded= tf.keras.layers.BatchNormalization()(decoded)
decoded = Dropout(0.5)(decoded)
decoded = Dense(1024, activation='relu')(decoded)
decoded= tf.keras.layers.BatchNormalization()(decoded)
decoded = Dropout(0.2)(decoded)
decoded = Dense(4096)(decoded)
# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

In [ ]:
#  this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

In [ ]:
autoencoder.compile(optimizer='adam', loss='mse',run_eagerly=True)

In [ ]:
train_history = autoencoder.fit(np.asarray(fts),np.asarray(lbls), epochs=100, batch_size=1000)

Epoch 1/100
1/1 [==============================] - 0s 143ms/step - loss: 2.2421
Epoch 2/100
1/1 [==============================] - 0s 111ms/step - loss: 2.2023
Epoch 3/100
1/1 [==============================] - 0s 98ms/step - loss: 2.0978
Epoch 4/100
1/1 [==============================] - 0s 81ms/step - loss: 1.9938
Epoch 5/100
1/1 [==============================] - 0s 78ms/step - loss: 1.9013
Epoch 6/100
1/1 [==============================] - 0s 76ms/step - loss: 1.8665
Epoch 7/100
1/1 [==============================] - 0s 76ms/step - loss: 1.7946
Epoch 8/100
1/1 [==============================] - 0s 81ms/step - loss: 1.7441
Epoch 9/100
1/1 [==============================] - 0s 82ms/step - loss: 1.6826
Epoch 10/100
1/1 [==============================] - 0s 80ms/step - loss: 1.6365
Epoch 11/100
1/1 [==============================] - 0s 78ms/step - loss: 1.6052
Epoch 12/100
1/1 [==============================] - 0s 80ms/step - loss: 1.5744
Epoch 13/100
1/1 [=============================

In [ ]:
pred_auto_train = encoder.predict(np.asarray(fts))

In [ ]:
pred_auto_train.shape

(1000, 1024)

In [ ]:
#convert resnet and densenet to df and save it as csv, om

# ***GCNN***

In [ ]:
from __future__ import print_function

import scipy.sparse as sp
import numpy as np
from scipy.sparse.linalg import eigsh, ArpackNoConvergence


def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot



def normalize_adj_numpy(adj, symmetric=True):
    if symmetric:
        d = np.diag(np.power(np.array(adj.sum(1)), -0.5).flatten(), 0)
        a_norm = adj.dot(d).transpose().dot(d)
    else:
        d = np.diag(np.power(np.array(adj.sum(1)), -1).flatten(), 0)
        a_norm = d.dot(adj)
    return a_norm



def preprocess_adj_numpy(adj, symmetric=True):
    adj = adj + np.eye(adj.shape[0])
    adj = normalize_adj_numpy(adj, symmetric)
    return adj




def sample_mask(idx, l):
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)



def categorical_crossentropy(preds, labels):
    return np.mean(-np.log(np.extract(labels, preds)))


def accuracy(preds, labels):
    return np.mean(np.equal(np.argmax(labels, 1), np.argmax(preds, 1)))


def evaluate_preds(preds, labels, indices):
    split_loss = list()
    split_acc = list()

    for y_split, idx_split in zip(labels, indices):
        split_loss.append(categorical_crossentropy(preds[idx_split], y_split[idx_split]))
        split_acc.append(accuracy(preds[idx_split], y_split[idx_split]))

    return split_loss, split_acc




In [ ]:
import keras.backend as K
import tensorflow as tf


def graph_conv_op(x, num_filters, graph_conv_filters, kernel):

    if len(x.get_shape()) == 2:
        conv_op = K.dot(graph_conv_filters, x)
        conv_op = tf.split(conv_op, num_filters, axis=0)
        conv_op = K.concatenate(conv_op, axis=1)
    elif len(x.get_shape()) == 3:
        conv_op = K.batch_dot(graph_conv_filters, x)
        conv_op = tf.split(conv_op, num_filters, axis=1)
        conv_op = K.concatenate(conv_op, axis=2)
    else:
        raise ValueError('x must be either 2 or 3 dimension tensor'
                         'Got input shape: ' + str(x.get_shape()))

    conv_out = K.dot(conv_op, kernel)
    return conv_out

In [ ]:
from keras import activations, initializers, constraints
from keras import regularizers
import keras.backend as K
from keras.layers import Layer
import tensorflow as tf


class GraphCNN(Layer):

    def __init__(self,
                 output_dim,
                 num_filters,
                 graph_conv_filters,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(GraphCNN, self).__init__(**kwargs)

        self.output_dim = output_dim
        self.num_filters = num_filters
        if num_filters != int(graph_conv_filters.get_shape().as_list()[-2]/graph_conv_filters.get_shape().as_list()[-1]):
            raise ValueError('num_filters does not match with graph_conv_filters dimensions.')
        self.graph_conv_filters = graph_conv_filters

        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.kernel_initializer.__name__ = kernel_initializer
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

    def build(self, input_shape):

        self.input_dim = input_shape[-1]
        kernel_shape = (self.num_filters * self.input_dim, self.output_dim)

        self.kernel = self.add_weight(shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.output_dim,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None

        self.built = True

    def call(self, input):

        output = graph_conv_op(input, self.num_filters, self.graph_conv_filters, self.kernel)
        if self.use_bias:
            output = K.bias_add(output, self.bias)
        if self.activation is not None:
            output = self.activation(output)
        return output

    def compute_output_shape(self, input_shape):
        output_shape = (input_shape[0], self.output_dim)
        return output_shape

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'num_filters': self.num_filters,
            'graph_conv_filters': self.graph_conv_filters,
            'activation': activations.serialize(self.activation),
            'use_bias': self.use_bias,
            'kernel_initializer': initializers.serialize(self.kernel_initializer),
            'bias_initializer': initializers.serialize(self.bias_initializer),
            'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),
            'activity_regularizer': regularizers.serialize(self.activity_regularizer),
            'kernel_constraint': constraints.serialize(self.kernel_constraint),
            'bias_constraint': constraints.serialize(self.bias_constraint)
        }
        base_config = super(GraphCNN, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split


In [ ]:
idx = np.random.permutation(len(pred_auto_train))
x= pred_auto_train[idx]

In [ ]:
#y = np.asarray(label)[idx]
y = np.asarray(lbls)[idx]

In [ ]:
print(x.shape)

(1000, 1024)


In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
from sklearn.metrics import pairwise_distances
# DF_var=x
# dfe = distance_matrix(DF_var, DF_var)
   

In [ ]:
DF_var=x
dfe = pairwise_distances(DF_var, DF_var)
   

In [ ]:
print(dfe.shape)

(1000, 1000)


In [ ]:
dfe[0].shape

(1000,)

In [ ]:
import pickle
with open('/content/dfe.pkl', 'wb') as f:
    pickle.dump(dfe, f)

# ***Preparing the adjacency matrix***

In [ ]:
dfe

array([[0.       , 4.502288 , 3.4984035, ..., 4.89018  , 6.444012 ,
        4.9261394],
       [4.502288 , 0.       , 4.590748 , ..., 6.3296394, 4.196389 ,
        1.6154155],
       [3.4984035, 4.590748 , 0.       , ..., 3.306862 , 5.652736 ,
        5.2034235],
       ...,
       [4.89018  , 6.3296394, 3.306862 , ..., 0.       , 5.518348 ,
        7.036909 ],
       [6.444012 , 4.196389 , 5.652736 , ..., 5.518348 , 0.       ,
        5.0838904],
       [4.9261394, 1.6154155, 5.2034235, ..., 7.036909 , 5.0838904,
        0.       ]], dtype=float32)

In [ ]:
from tqdm import tqdm
df = dfe.copy()
a=np.asarray(df)
d=np.asarray(df)
# s=0
# for i in range(d.shape[0]):
#   for j in range(d.shape[1]):
#     s=s+d[i,j]
# mean = s/(d.shape[0]*d.shape[1])
mean = np.mean(d)


In [ ]:
mean

6.2646995

In [ ]:
df

array([[0.       , 4.502288 , 3.4984035, ..., 4.89018  , 6.444012 ,
        4.9261394],
       [4.502288 , 0.       , 4.590748 , ..., 6.3296394, 4.196389 ,
        1.6154155],
       [3.4984035, 4.590748 , 0.       , ..., 3.306862 , 5.652736 ,
        5.2034235],
       ...,
       [4.89018  , 6.3296394, 3.306862 , ..., 0.       , 5.518348 ,
        7.036909 ],
       [6.444012 , 4.196389 , 5.652736 , ..., 5.518348 , 0.       ,
        5.0838904],
       [4.9261394, 1.6154155, 5.2034235, ..., 7.036909 , 5.0838904,
        0.       ]], dtype=float32)

In [ ]:
d

array([[0.       , 4.502288 , 3.4984035, ..., 4.89018  , 6.444012 ,
        4.9261394],
       [4.502288 , 0.       , 4.590748 , ..., 6.3296394, 4.196389 ,
        1.6154155],
       [3.4984035, 4.590748 , 0.       , ..., 3.306862 , 5.652736 ,
        5.2034235],
       ...,
       [4.89018  , 6.3296394, 3.306862 , ..., 0.       , 5.518348 ,
        7.036909 ],
       [6.444012 , 4.196389 , 5.652736 , ..., 5.518348 , 0.       ,
        5.0838904],
       [4.9261394, 1.6154155, 5.2034235, ..., 7.036909 , 5.0838904,
        0.       ]], dtype=float32)

In [ ]:
# for i in tqdm(range(d.shape[0])):
#   for j in range(d.shape[1]):
#     if(d[i,j]>=mean):
#       a[i,j]=1
#     else:
#       a[i,j]=0

d[d < mean] = 0
d[d >= mean] = 1


In [ ]:
d

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [ ]:
A=d
A.shape

(1000, 1000)

In [ ]:
A

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [ ]:
# Build Graph Convolution filters
SYM_NORM = True
A_norm = preprocess_adj_numpy(A, SYM_NORM)
num_filters = 2


In [ ]:
graph_conv_filters = np.concatenate([A_norm, np.matmul(A_norm, A_norm)], axis=0)
graph_conv_filters = K.constant(graph_conv_filters)

In [ ]:
with open('graph_conv_filters.pkl','wb') as f:
  pickle.dump(graph_conv_filters,f)

# ***Building and fitting on the graph convolution network***

In [ ]:
A_hat = A_norm.copy()

In [ ]:
A

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [ ]:
A_norm.shape

(1000, 1000)

In [ ]:
A_norm

array([[0.00202429, 0.        , 0.        , ..., 0.        , 0.00265119,
        0.        ],
       [0.        , 0.00275482, 0.        , ..., 0.00326766, 0.        ,
        0.        ],
       [0.        , 0.        , 0.00378788, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.00326766, 0.        , ..., 0.00387597, 0.        ,
        0.00296463],
       [0.00265119, 0.        , 0.        , ..., 0.        , 0.00347222,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00296463, 0.        ,
        0.00226757]])

In [ ]:
with open('A_norm.pkl','wb') as f:
  pickle.dump(A_norm,f)

In [ ]:
X=x
Y=y
X_train = np.zeros((MAX_LEN,1024))
X_test =np.zeros((MAX_LEN,1024))
y_train=np.zeros(MAX_LEN)
y_test = np.zeros(MAX_LEN)

SPLIT = 800

X_train[:SPLIT] =x[:SPLIT]
y_train[:SPLIT] = y[:SPLIT]
X_test[SPLIT:]=x[SPLIT:]
y_test[SPLIT:]=y[SPLIT:]
num_filters=2


In [ ]:
A.shape[0] 

1000

In [ ]:
X.shape[0]

1000

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1000, 1024)
(1000, 1024)
(1000,)
(1000,)


In [ ]:
# Build Model 
from keras.layers import Input
from keras.models import Sequential

model = Sequential()
model.add(Input(shape=(1024,)))

model.add(Dense(512,activation='relu'))
model.add(GraphCNN(512, num_filters, graph_conv_filters,  activation='relu'))
model.add(Activation('relu'))
model.add(Dropout(0.2))

# model.add(Dense(128,activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# model.add(Dense(64,activation='sigmoid'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# model.add(Dense(128,activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

model.add(Dense(32,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(16,activation='relu'))
model.add(Dense(3,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['acc'])
model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_59 (Dense)            (None, 512)               524800    
                                                                 
 graph_cnn_6 (GraphCNN)      (1000, 512)               524800    
                                                                 
 activation_6 (Activation)   (1000, 512)               0         
                                                                 
 dropout_39 (Dropout)        (1000, 512)               0         
                                                                 
 dense_60 (Dense)            (1000, 32)                16416     
                                                                 
 batch_normalization_33 (Bat  (1000, 32)               128       
 chNormalization)                                                
                                                      

In [ ]:
# tf.config.run_functions_eagerly(True)
# tf.config.experimental_run_functions_eagerly(True)

In [ ]:
A.shape

(1000, 1000)

In [ ]:
nb_epochs = 1000
y2 = to_categorical(y_train)
BATCH_SIZE = A.shape[0]
print(y2.shape)

(1000, 3)


In [ ]:
for epoch in range(nb_epochs):
    model.fit(X_train,y2, batch_size=BATCH_SIZE, epochs=10, shuffle=False, verbose=0)
    Y_pred1 = model.predict(X_train, batch_size=BATCH_SIZE)
    #Y_pred2 = model.predict(X_test, batch_size=A.shape[0])

    train_acc = accuracy_score(np.argmax(Y_pred1[0:SPLIT],axis=1),y_train[0:SPLIT])
    test_acc = accuracy_score(np.argmax(Y_pred1[SPLIT:],axis=1),y_test[SPLIT:])
    print("Epoch: {:04d}".format(epoch+1), "train_acc = {:.4f}".format(train_acc), "test_acc = {:.4f}".format(test_acc))

Epoch: 0001 train_acc = 0.3250 test_acc = 0.2800
Epoch: 0002 train_acc = 0.4850 test_acc = 0.4900
Epoch: 0003 train_acc = 0.3463 test_acc = 0.3800
Epoch: 0004 train_acc = 0.3463 test_acc = 0.3800
Epoch: 0005 train_acc = 0.3463 test_acc = 0.3800
Epoch: 0006 train_acc = 0.3463 test_acc = 0.3800
Epoch: 0007 train_acc = 0.3463 test_acc = 0.3800
Epoch: 0008 train_acc = 0.3463 test_acc = 0.3800
Epoch: 0009 train_acc = 0.3463 test_acc = 0.3800
Epoch: 0010 train_acc = 0.3488 test_acc = 0.3800
Epoch: 0011 train_acc = 0.4175 test_acc = 0.4450
Epoch: 0012 train_acc = 0.5437 test_acc = 0.5250
Epoch: 0013 train_acc = 0.3488 test_acc = 0.3000
Epoch: 0014 train_acc = 0.3250 test_acc = 0.2800
Epoch: 0015 train_acc = 0.3250 test_acc = 0.2800
Epoch: 0016 train_acc = 0.3250 test_acc = 0.2800
Epoch: 0017 train_acc = 0.3250 test_acc = 0.2800
Epoch: 0018 train_acc = 0.3250 test_acc = 0.2800
Epoch: 0019 train_acc = 0.3250 test_acc = 0.2800
Epoch: 0020 train_acc = 0.3250 test_acc = 0.2800
Epoch: 0021 train_ac

In [ ]:
final_train_acc = accuracy_score(np.argmax(Y_pred1[0:SPLIT],axis=1),y_train[0:SPLIT])
final_test_acc = accuracy_score(np.argmax(Y_pred1[SPLIT:],axis=1),y_test[SPLIT:])

In [ ]:
print("the accuracy on training set is", final_train_acc)
print("the accuracy on test set is", final_test_acc)

the accuracy on training set is 0.98
the accuracy on test set is 0.975


In [ ]:
with open('A_norm.pkl','rb') as f:
  anm = pickle.load(f)

In [ ]:
with open('A_NORM.pkl','wb') as f:
  pickle.dump(anm,f)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(np.argmax(Y_pred1[SPLIT:],axis=1),y_test[SPLIT:]))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97        55
           1       0.97      0.96      0.96        69
           2       0.99      0.99      0.99        76

    accuracy                           0.97       200
   macro avg       0.97      0.98      0.97       200
weighted avg       0.98      0.97      0.97       200

